In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [2]:
sparql = SPARQLWrapper("https://landregistry.data.gov.uk/landregistry/query")
sparql.setReturnFormat(JSON)

In [11]:
# gets the first 3 geological ages
# from a Geological Timescale database,
# via a SPARQL endpoint
sparql.setQuery("""

prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix owl: <http://www.w3.org/2002/07/owl#>
prefix xsd: <http://www.w3.org/2001/XMLSchema#>
prefix sr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
prefix ukhpi: <http://landregistry.data.gov.uk/def/ukhpi/>
prefix lrppi: <http://landregistry.data.gov.uk/def/ppi/>
prefix skos: <http://www.w3.org/2004/02/skos/core#>
prefix lrcommon: <http://landregistry.data.gov.uk/def/common/>

# Returns house price index average prices for flats, semis, detacted and terraced for all LA since 2015, with GSS codes

SELECT
  ?regionName ?code ?date ?hpi ?hpiDetached ?hpiFlatMaisonette ?hpiSemiDetached ?hpiTerraced ?averagePriceDetached ?averagePriceFlatMaisonette ?averagePriceSemiDetached ?averagePriceTerraced
WHERE
{

  ?x
    ukhpi:refRegion ?region;
    ukhpi:refMonth ?date;
    ukhpi:housePriceIndex ?hpi;
    ukhpi:housePriceIndexFlatMaisonette ?hpiFlatMaisonette;
    ukhpi:averagePriceFlatMaisonette ?averagePriceFlatMaisonette;
    ukhpi:refPeriodStart ?startdate.
  
  OPTIONAL{
    ?x ukhpi:housePriceIndexDetached ?hpiDetached;
    ukhpi:housePriceIndexSemiDetached ?hpiSemiDetached;
    ukhpi:housePriceIndexTerraced ?hpiTerraced;
    ukhpi:averagePriceSemiDetached ?averagePriceSemiDetached;
    ukhpi:averagePriceTerraced ?averagePriceTerraced;
    ukhpi:averagePriceDetached ?averagePriceDetached.
    }

  ?region owl:sameAs ?code.
#  ?region rdfs:seeAlso ?codey #can also use this one
  ?region rdfs:label ?regionName .
# FILTER regex(str(?code), "statistics") #this also works
  FILTER contains(str(?code),"gov")
  FILTER (langMatches( lang(?regionName), "EN")&&
         ?startdate > "2015-12-31"^^xsd:date)
}
    """
)

try:
    ret = sparql.queryAndConvert()

#     for r in ret["results"]["bindings"]:
#         print(r)
except Exception as e:
    print(e)

pd.json_normalize(ret["results"]["bindings"])

,regionName.type,regionName.xml:lang,regionName.value,code.type,code.value,date.type,date.datatype,date.value,hpi.type,hpi.datatype,...,averagePriceDetached.value,averagePriceFlatMaisonette.type,averagePriceFlatMaisonette.datatype,averagePriceFlatMaisonette.value,averagePriceSemiDetached.type,averagePriceSemiDetached.datatype,averagePriceSemiDetached.value,averagePriceTerraced.type,averagePriceTerraced.datatype,averagePriceTerraced.value
0,literal,en,Kensington and Chelsea,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2016-08,literal,http://www.w3.org/2001/XMLSchema#decimal,...,3105572,literal,http://www.w3.org/2001/XMLSchema#integer,1058127,literal,http://www.w3.org/2001/XMLSchema#integer,3100329,literal,http://www.w3.org/2001/XMLSchema#integer,2169487
1,literal,en,Kensington and Chelsea,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2019-10,literal,http://www.w3.org/2001/XMLSchema#decimal,...,3331990,literal,http://www.w3.org/2001/XMLSchema#integer,1024712,literal,http://www.w3.org/2001/XMLSchema#integer,3127042,literal,http://www.w3.org/2001/XMLSchema#integer,2147854
2,literal,en,Kensington and Chelsea,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2019-11,literal,http://www.w3.org/2001/XMLSchema#decimal,...,3220623,literal,http://www.w3.org/2001/XMLSchema#integer,1021388,literal,http://www.w3.org/2001/XMLSchema#integer,3127858,literal,http://www.w3.org/2001/XMLSchema#integer,2143720
3,literal,en,Kensington and Chelsea,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2016-10,literal,http://www.w3.org/2001/XMLSchema#decimal,...,3125850,literal,http://www.w3.org/2001/XMLSchema#integer,1068270,literal,http://www.w3.org/2001/XMLSchema#integer,3129562,literal,http://www.w3.org/2001/XMLSchema#integer,2182776
4,literal,en,Kensington and Chelsea,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2016-07,literal,http://www.w3.org/2001/XMLSchema#decimal,...,3155487,literal,http://www.w3.org/2001/XMLSchema#integer,1078930,literal,http://www.w3.org/2001/XMLSchema#integer,3139360,literal,http://www.w3.org/2001/XMLSchema#integer,2202894
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14935,literal,en,Manchester,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2022-09,literal,http://www.w3.org/2001/XMLSchema#decimal,...,435269,literal,http://www.w3.org/2001/XMLSchema#integer,193238,literal,http://www.w3.org/2001/XMLSchema#integer,298830,literal,http://www.w3.org/2001/XMLSchema#integer,225510
14936,literal,en,Salford,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2022-11,literal,http://www.w3.org/2001/XMLSchema#decimal,...,407950,literal,http://www.w3.org/2001/XMLSchema#integer,166969,literal,http://www.w3.org/2001/XMLSchema#integer,264895,literal,http://www.w3.org/2001/XMLSchema#integer,195987
14937,literal,en,Tameside,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2022-10,literal,http://www.w3.org/2001/XMLSchema#decimal,...,370488,literal,http://www.w3.org/2001/XMLSchema#integer,132433,literal,http://www.w3.org/2001/XMLSchema#integer,236763,literal,http://www.w3.org/2001/XMLSchema#integer,178838
14938,literal,en,Manchester,uri,http://statistics.data.gov.uk/id/statistical-g...,literal,http://www.w3.org/2001/XMLSchema#gYearMonth,2022-11,literal,http://www.w3.org/2001/XMLSchema#decimal,...,441483,literal,http://www.w3.org/2001/XMLSchema#integer,194585,literal,http://www.w3.org/2001/XMLSchema#integer,302060,literal,http://www.w3.org/2001/XMLSchema#integer,227976


In [12]:
df=pd.json_normalize(ret["results"]["bindings"])

In [13]:
df['code']=df['code.value'].str.split('/',expand=True)[5]

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14940 entries, 0 to 14939
Data columns (total 36 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   regionName.type                      14940 non-null  object
 1   regionName.xml:lang                  14940 non-null  object
 2   regionName.value                     14940 non-null  object
 3   code.type                            14940 non-null  object
 4   code.value                           14940 non-null  object
 5   date.type                            14940 non-null  object
 6   date.datatype                        14940 non-null  object
 7   date.value                           14940 non-null  object
 8   hpi.type                             14940 non-null  object
 9   hpi.datatype                         14940 non-null  object
 10  hpi.value                            14940 non-null  object
 11  hpiDetached.type                     1492

In [15]:
df2=df[['regionName.value', 'code','date.value','hpi.value','hpiDetached.value','hpiFlatMaisonette.value','hpiSemiDetached.value','hpiTerraced.value', 'averagePriceDetached.value','averagePriceFlatMaisonette.value','averagePriceSemiDetached.value','averagePriceTerraced.value' ]]

In [16]:
df2

,regionName.value,code,date.value,hpi.value,hpiDetached.value,hpiFlatMaisonette.value,hpiSemiDetached.value,hpiTerraced.value,averagePriceDetached.value,averagePriceFlatMaisonette.value,averagePriceSemiDetached.value,averagePriceTerraced.value
0,Kensington and Chelsea,E09000020,2016-08,91.89,89.84,92.52,88.16,89.01,3105572,1058127,3100329,2169487
1,Kensington and Chelsea,E09000020,2019-10,89.48,96.39,89.60,88.92,88.12,3331990,1024712,3127042,2147854
2,Kensington and Chelsea,E09000020,2019-11,89.20,93.17,89.31,88.94,87.95,3220623,1021388,3127858,2143720
3,Kensington and Chelsea,E09000020,2016-10,92.71,90.43,93.41,88.99,89.55,3125850,1068270,3129562,2182776
4,Kensington and Chelsea,E09000020,2016-07,93.62,91.29,94.34,89.27,90.38,3155487,1078930,3139360,2202894
...,...,...,...,...,...,...,...,...,...,...,...,...
14935,Manchester,E08000003,2022-09,174.68,186.78,158.91,185.67,181.43,435269,193238,298830,225510
14936,Salford,E08000006,2022-11,178.36,189.48,165.63,186.72,181.47,407950,166969,264895,195987
14937,Tameside,E08000008,2022-10,183.25,188.44,161.03,186.80,181.83,370488,132433,236763,178838
14938,Manchester,E08000003,2022-11,176.40,189.44,160.02,187.68,183.42,441483,194585,302060,227976


In [17]:
df2.to_csv('landreg.csv',index=False)